In [1]:
import pandas as pd
import numpy as np
import nltk as nltk
import spacy
import regex
import re
import sklearn
from nltk import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

### Read in the data

In [38]:
train_data_path = "Data/webis-clickbait-22/train.jsonl"
df = pd.read_json(train_data_path, lines=True)

In [39]:
pd.set_option('display.max_colwidth', None)

### Select important columns for analysis:

In [40]:
df.head()

uuid              postId  \
0  0af11f6b-c889-4520-9372-66ba25cb7657              532quh   
1  b1a1f63d-8853-4a11-89e8-6b2952a393ec  411701128456593408   
2  008b7b19-0445-4e16-8f9e-075b73f80ca4  380537005123190784   
3  31ecf93c-3e21-4c80-949b-aa549a046b93  844567852531286016   
4  31b108a3-c828-421a-a4b9-cf651e9ac859  814186311573766144   

                                                                                   postText  \
0                [Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea]   
1                                          [NASA sets date for full recovery of ozone hole]   
2                       [This is what makes employees happy -- and it's not their paycheck]   
3                                   [Passion is overrated — 7 work habits you need instead]   
4  [The perfect way to cook rice so that it's perfectly fluffy and NEVER sticks to the pan]   

  postPlatform  \
0       reddit   
1      Twitter   
2      Twitter   
3      Twitter   
4      Twitter   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

We are using the Posttext, Targetparagraph, spoiler and the postplatform to possibly classify the spoiler type (tag)

In [41]:
df_selected = df[['postText','targetParagraphs', 'spoiler', 'postPlatform', 'tags']]

# convert all columns into strings
df_selected[['postPlatform', 'postText', 'targetParagraphs', 'spoiler', 'tags']] = df_selected[['postPlatform','postText', 'targetParagraphs', 'spoiler', 'tags']].astype(str)

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [42]:
df_selected.head()

postText  \
0                ['Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea']   
1                                          ['NASA sets date for full recovery of ozone hole']   
2                       ["This is what makes employees happy -- and it's not their paycheck"]   
3                                ['Passion is overrated —\xa07 work habits you need instead']   
4  ["The perfect way to cook rice so that it's perfectly fluffy and NEVER sticks to the pan"]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

### Text is cleaned and pre-processed in the following steps:

In [43]:
tokenizer = RegexpTokenizer(r"\w+")

In [44]:
stopwords = nltk.corpus.stopwords.words("english")

In [45]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

In [46]:
def preprocess_data(df):
    df = df[['postText','targetParagraphs', 'spoiler', 'postPlatform', 'tags']]

# convert all columns into strings
    df[['postPlatform', 'postText', 'targetParagraphs', 'spoiler', 'tags']] = df[['postPlatform','postText', 'targetParagraphs', 'spoiler', 'tags']].astype(str)
    tokenizer = RegexpTokenizer(r"\w+")
    df["postText_tokens"] = df.apply(lambda row: tokenizer.tokenize(row["postText"]), axis = 1)
    df["paragraph_tokens"] = df.apply(lambda row: tokenizer.tokenize(row["targetParagraphs"]), axis = 1)
    df["spoiler_tokens"] = df.apply(lambda row: tokenizer.tokenize(row["spoiler"]), axis = 1)
    
    stopwords = nltk.corpus.stopwords.words("english")

    df["postText_tokens"] = df.apply(lambda row: [element for element in row["postText_tokens"] if element not in stopwords], axis = 1)
    df["paragraph_tokens"] = df.apply(lambda row: [element for element in row["paragraph_tokens"] if element not in stopwords], axis = 1)
    df["spoiler_tokens"] = df.apply(lambda row: [element for element in row["spoiler_tokens"] if element not in stopwords], axis = 1)
    
    df['postText_tokens'] = df['postText_tokens'].map(lambda row: list(map(str.lower, row)))
    df['paragraph_tokens'] = df['paragraph_tokens'].map(lambda row: list(map(str.lower, row)))
    df['spoiler_tokens'] = df['spoiler_tokens'].map(lambda row: list(map(str.lower, row)))
    
    # multiple space to single space
    df[['postText_tokens', 'paragraph_tokens', 'spoiler_tokens']] = df[['postText_tokens', 'paragraph_tokens', 'spoiler_tokens']].replace(r'\s+', ' ', regex=True)
    #special characters
    df[['postText_tokens', 'paragraph_tokens', 'spoiler_tokens']] = df[['postText_tokens', 'paragraph_tokens', 'spoiler_tokens']].replace(r'\W', ' ', regex = True)

    df['postText_tokens'] = df['postText_tokens'].apply(lemmatize_text)
    df['paragraph_tokens'] = df['paragraph_tokens'].apply(lemmatize_text)
    df['spoiler_tokens'] = df['spoiler_tokens'].apply(lemmatize_text)
    
    df['postText_length'] = ""
    df['paragraph_length'] = ""
    df['spoiler_length'] = ""
    for i in range(len(df)):
        df['postText_length'][i] = len(df['postText'][i])
        df['paragraph_length'][i] = len(df['targetParagraphs'][i])
        df['spoiler_length'][i] = len(df['spoiler'][i])
    
    for i in range(0, len(df)):
        questionmark = "?"
        df['has_questionmark'] = 'posthasquestionmark'
        if questionmark in df['postText'][i]:
            df['has_questionmark'] = 'posthasnoquestionmark'
    return df


In [47]:
df_selected = preprocess_data(df)

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_10267/4112907407.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["postText_tokens"] = df.apply(lambda row: tokenizer.tokenize(row["postText"]), axis = 1)
/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_10267/4112907407.py:8: SettingWithCopyWarning: 
A value is trying to be set

/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_10267/4112907407.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['has_questionmark'] = 'posthasquestionmark'
/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_10267/4112907407.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['has_questionmark'] = 'posthasnoquestionmark'


In [48]:
#currently hardcoded
df_selected['postText_length'] = df_selected['postText_length'].apply(lambda x: 'overavg_post_length' if x > 33 else 'underavg_post_length')
df_selected['paragraph_length'] = df_selected['paragraph_length'].apply(lambda x: 'overavg_paragraph_length' if x > 1534 else 'underavg_paragraph_length')
df_selected['spoiler_length'] = df_selected['spoiler_length'].apply(lambda x: 'overavg_spoiler_length' if x > 49 else 'underavg_spoiler_length')


/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_10267/1090028342.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['postText_length'] = df_selected['postText_length'].apply(lambda x: 'overavg_post_length' if x > 33 else 'underavg_post_length')
/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_10267/1090028342.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['paragraph_length'] = df_selected['paragraph_length'].apply(lambda x: 'overavg_paragraph_length' if x >

In [49]:
for i in range(len(df_selected)):
    df_selected['has_numeric'] = any(str.isdigit(c) for c in df_selected['targetParagraphs'][i])
    
df_selected['has_numeric'] = np.where(df_selected['has_numeric'], 'hasnumeric', 'nonumeric')

/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_10267/684289240.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['has_numeric'] = any(str.isdigit(c) for c in df_selected['targetParagraphs'][i])
/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_10267/684289240.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['has_numeric'] = np.where(df_selected['has_numeric'], 'hasnumeric', 'nonumeric')


In [50]:
nlp = spacy.load('en_core_web_lg')
df_selected['Entities'] = df_selected['postText'].apply(lambda sent: [(ent.text, ent.label_) for ent in nlp(sent).ents])  
df_selected['Entities'][0]

[('Wes Welker', 'PERSON'), ('Tom Brady', 'PERSON'), ('Patriots QB', 'ORG')]

In [51]:
for i in range(len(df_selected)):
    tostring = str(df_selected['Entities'][i])
    tostring = ' '.join(str(item) for tup in df_selected['Entities'][i] for item in tup)
    #print(tostring)
    df_selected['Entities'][i] = tostring
  

In [52]:
df_selected['combined_texts'] = df_selected['postText'] + " " + df_selected['targetParagraphs'] + " " + df_selected['postText_length'] + " " + df_selected['paragraph_length'] + " " + df_selected['has_questionmark'] + " " + df_selected['has_numeric'] + " " + df_selected['postPlatform'] + " " + df_selected['Entities']

In [53]:
df_selected = df_selected#[:2500]

In [54]:
y = df_selected['tags']

### Convert our text into bag of word vectors


In [26]:
#?vectorizer

In [27]:
#max_features could be used to constrain the amount of words
# ngram_range 2,2 is too long
#ngram range 1,2 works without tf-idf
vectorizer = CountVectorizer(stop_words=stopwords, lowercase=True, tokenizer = tokenizer.tokenize, ngram_range=(1,1))

In [28]:
# Add the vectorized dataframes to another and transform them back
combined_bow = vectorizer.fit_transform(df_selected['combined_texts']).toarray()
bag_of_words = pd.DataFrame(combined_bow, columns=vectorizer.get_feature_names())

In [29]:
bag_of_words

,0,00,000,00000000000000000000000000000000000000625,00000000130,000001002003004005006,0000024,0005,000s,001,...,いいね,コメント,ツ,件,寵物小精靈,比卡超,皮卡丘,神奇寶貝,精靈寶可夢,認証済み
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
X = combined_bow

In [31]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### Finding TFIDF of our bag of words

(n = 1,2) took over an hour and didnt compute.

In [32]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()

In [33]:
X = tfidfconverter.fit_transform(X).toarray()

In [34]:
X = pd.DataFrame(X)

In [35]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200 entries, 0 to 3199
Columns: 55614 entries, 0 to 55613
dtypes: float64(55614)
memory usage: 1.3 GB


In [36]:
X

,0,1,2,3,4,5,6,7,8,9,...,55604,55605,55606,55607,55608,55609,55610,55611,55612,55613
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.014808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3196,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3197,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3198,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Apply ML techniques

- NER etc. 
- text lengths -> Added
- count has numbers
- count new lines
- check if numbers are followed by dots or strings
- 


**Categorical to numeric**:

multi = 0, passage = 1, phrase = 2

**ngrams of 1 and 2, 1000 rows: **

RF: 0.485

logreg: 0.47

bayes: 0.395

**ngrams of 2, 1500 rows:**

similar, however only phrases are classified.

**ngrams of 1, 3200 rows:**

RF: 0.46

logreg: 0.5

bayes: 0.39

**ngrams of 1 and 2 without tf-idf:**

RF: 0.52

logreg: 0.56

bayes: 0.41

rf generally better with bigrams

rf seems better without tf-idf, logreg not

### Random Forest

In [55]:
y = pd.DataFrame(y)
y = pd.get_dummies(y['tags'])
y = y.values.argmax(1)
y = pd.DataFrame(y)
y = y.rename(columns={0: "tags"})

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)

In [57]:
classifier = RandomForestClassifier(n_estimators=100, random_state=8, max_depth=50)
classifier.fit(X_train, y_train) 

/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_10267/2341424750.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


RandomForestClassifier(max_depth=50, random_state=8)

In [58]:
y_pred = classifier.predict(X_test)

In [59]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[  8  51  48]
 [  0 117 138]
 [  0  83 195]]
              precision    recall  f1-score   support

           0       1.00      0.07      0.14       107
           1       0.47      0.46      0.46       255
           2       0.51      0.70      0.59       278

    accuracy                           0.50       640
   macro avg       0.66      0.41      0.40       640
weighted avg       0.58      0.50      0.46       640

0.5


### Logistic Regression

In [44]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

In [45]:
y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)

In [46]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[ 15  44  48]
 [  1 128 126]
 [  6  89 183]]
              precision    recall  f1-score   support

           0       0.68      0.14      0.23       107
           1       0.49      0.50      0.50       255
           2       0.51      0.66      0.58       278

    accuracy                           0.51       640
   macro avg       0.56      0.43      0.44       640
weighted avg       0.53      0.51      0.49       640

0.509375


### Naive Bayes

In [47]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [48]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[ 21  44  42]
 [ 52 114  89]
 [ 57  98 123]]
              precision    recall  f1-score   support

           0       0.16      0.20      0.18       107
           1       0.45      0.45      0.45       255
           2       0.48      0.44      0.46       278

    accuracy                           0.40       640
   macro avg       0.36      0.36      0.36       640
weighted avg       0.41      0.40      0.41       640

0.403125


### XgBoost

In [49]:
import xgboost as xgb

In [50]:
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [51]:
params = { "objective": "multi:softmax", 'num_class': 3, 'max_depth' : 10}

In [52]:
d_train = xgb.DMatrix(X_train, y_train)
d_test = xgb.DMatrix(X_test, y_test)

In [53]:
xgb_model = xgb.train(params, d_train, num_boost_round=5)

In [54]:
y_pred = xgb_model.predict(d_test)
#y_pred = np.where(np.array(y_pred) > 0.5, 1, 0)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[ 24  42  41]
 [ 13 128 114]
 [ 13 105 160]]
              precision    recall  f1-score   support

           0       0.48      0.22      0.31       107
           1       0.47      0.50      0.48       255
           2       0.51      0.58      0.54       278

    accuracy                           0.49       640
   macro avg       0.48      0.43      0.44       640
weighted avg       0.49      0.49      0.48       640

0.4875


### KNearestNeighbors

In [55]:
from sklearn.neighbors import KNeighborsClassifier

In [56]:
model = KNeighborsClassifier(n_neighbors = 3)
model.fit(X_train, y_train)

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=3)

In [57]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[ 47  26  34]
 [ 65 104  86]
 [ 98  82  98]]
              precision    recall  f1-score   support

           0       0.22      0.44      0.30       107
           1       0.49      0.41      0.45       255
           2       0.45      0.35      0.40       278

    accuracy                           0.39       640
   macro avg       0.39      0.40      0.38       640
weighted avg       0.43      0.39      0.40       640

0.3890625
